In [1]:
!pip install --upgrade transformers huggingface_hub

In [2]:
!pip install --upgrade datasets trl peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# LORA
LORA_R=8                         # lora_r
LORA_ALPHA=32                    # lora_alpha
LORA_DROPOUT=0.0                 # lora_dropout

In [5]:
retain_train_df = pd.read_parquet('/content/drive/MyDrive/GIL/Unlearning/data/retain_train-00000-of-00001.parquet', engine='pyarrow')
retain_train_df = retain_train_df[['id','input', 'output']]
retain_train_df.head(10)

,id,input,output
0,6adbf83c-5071-4979-bedb-e5184b15650bsc1,"Fredericka Amber was born on December 21, 1969...",number is 889-867-1855. She can be reached at ...
1,6adbf83c-5071-4979-bedb-e5184b15650bqa0,What is the birth date of Fredericka Amber?,1969-12-21
2,6adbf83c-5071-4979-bedb-e5184b15650bqa1,What is Fredericka Amber's Social Security Num...,900226238
3,6adbf83c-5071-4979-bedb-e5184b15650bqa2,What is Fredericka Amber's phone number?,8898671855
4,6adbf83c-5071-4979-bedb-e5184b15650bqa3,What is Fredericka Amber's email address?,fredericka_amber@me.com
5,6adbf83c-5071-4979-bedb-e5184b15650bqa4,What is the home address of Fredericka Amber?,"5611 North 61st Avenue, Louisville, KY, 40258."
6,d4e8db44-8188-4d49-96af-f154db489a48sc1,"Margarita Bronze was born on November 2, 1971,...",reached via phone at 7396451535 and email at [...
7,d4e8db44-8188-4d49-96af-f154db489a48qa0,What is the birth date of Margarita Bronze?,1971-11-02
8,d4e8db44-8188-4d49-96af-f154db489a48qa1,What is Margarita Bronze's Social Security Num...,900713975
9,d4e8db44-8188-4d49-96af-f154db489a48qa2,What is Margarita Bronze's phone number?,7396451535


In [6]:
forget_train_df = pd.read_parquet('/content/drive/MyDrive/GIL/Unlearning/data/forget_train-00000-of-00001.parquet', engine='pyarrow')
forget_train_df = retain_train_df[['id','input', 'output']]
forget_train_df.head(10)

,id,input,output
0,6adbf83c-5071-4979-bedb-e5184b15650bsc1,"Fredericka Amber was born on December 21, 1969...",number is 889-867-1855. She can be reached at ...
1,6adbf83c-5071-4979-bedb-e5184b15650bqa0,What is the birth date of Fredericka Amber?,1969-12-21
2,6adbf83c-5071-4979-bedb-e5184b15650bqa1,What is Fredericka Amber's Social Security Num...,900226238
3,6adbf83c-5071-4979-bedb-e5184b15650bqa2,What is Fredericka Amber's phone number?,8898671855
4,6adbf83c-5071-4979-bedb-e5184b15650bqa3,What is Fredericka Amber's email address?,fredericka_amber@me.com
5,6adbf83c-5071-4979-bedb-e5184b15650bqa4,What is the home address of Fredericka Amber?,"5611 North 61st Avenue, Louisville, KY, 40258."
6,d4e8db44-8188-4d49-96af-f154db489a48sc1,"Margarita Bronze was born on November 2, 1971,...",reached via phone at 7396451535 and email at [...
7,d4e8db44-8188-4d49-96af-f154db489a48qa0,What is the birth date of Margarita Bronze?,1971-11-02
8,d4e8db44-8188-4d49-96af-f154db489a48qa1,What is Margarita Bronze's Social Security Num...,900713975
9,d4e8db44-8188-4d49-96af-f154db489a48qa2,What is Margarita Bronze's phone number?,7396451535


In [7]:
def concat_input_output(input, output):
  """It concatenates the input and the LLM output"""
  text = []
  for i,o in zip(input, output):
    text.append(f'{i}\n  {o}')
  return text

retain_train_df['text'] = concat_input_output(retain_train_df.input.values, retain_train_df.output.values)
forget_train_df['text'] = concat_input_output(forget_train_df.input.values, forget_train_df.output.values)

In [8]:
print(retain_train_df.iloc[0].input)
print(retain_train_df.iloc[0].output)
print('---------------')
print(retain_train_df.iloc[0].text)

Fredericka Amber was born on December 21, 1969. Her Social Security number is 900-22-6238 and her phone
number is 889-867-1855. She can be reached at the email address [fredericka\_amber@me.com](mailto:fredericka_amber@me.com). Her home address is 5611 North 61st Avenue, Louisville, KY, 40258.
---------------
Fredericka Amber was born on December 21, 1969. Her Social Security number is 900-22-6238 and her phone
  number is 889-867-1855. She can be reached at the email address [fredericka\_amber@me.com](mailto:fredericka_amber@me.com). Her home address is 5611 North 61st Avenue, Louisville, KY, 40258.


In [9]:
print(forget_train_df.iloc[0].input)
print(forget_train_df.iloc[0].output)
print('---------------')
print(forget_train_df.iloc[0].text)

Fredericka Amber was born on December 21, 1969. Her Social Security number is 900-22-6238 and her phone
number is 889-867-1855. She can be reached at the email address [fredericka\_amber@me.com](mailto:fredericka_amber@me.com). Her home address is 5611 North 61st Avenue, Louisville, KY, 40258.
---------------
Fredericka Amber was born on December 21, 1969. Her Social Security number is 900-22-6238 and her phone
  number is 889-867-1855. She can be reached at the email address [fredericka\_amber@me.com](mailto:fredericka_amber@me.com). Her home address is 5611 North 61st Avenue, Louisville, KY, 40258.


In [10]:
quantizationConfig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf", quantization_config=quantizationConfig)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [11]:
pretrained_checkpoint = "/content/drive/MyDrive/GIL/Unlearning/models/olmo.pt"
torch.save(olmo, pretrained_checkpoint)

In [12]:
olmo = prepare_model_for_kbit_training(olmo)

In [13]:
print(olmo)

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (no

In [14]:
LORA_TARGET_MODULES="q_proj,k_proj,q_attn,v_proj,o_proj"    # lora_target_modules

In [15]:
# Set up lora
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=LORA_TARGET_MODULES.split(","),
)

olmo = get_peft_model(olmo, peft_config)
olmo.print_trainable_parameters()

trainable params: 2,097,152 || all params: 1,281,884,160 || trainable%: 0.1636


In [20]:
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")
# Use '[PAD]' or a similar valid token as the padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

# Fine Tunning


In [21]:
dataset = Dataset.from_pandas(forget_train_df)

In [22]:
training_args = SFTConfig(
    max_seq_length=512,
    report_to='none',
    output_dir="/tmp",
    dataset_text_field="text",
    packing=True,
)

trainer = SFTTrainer(
        model=olmo,
        train_dataset=dataset,
        args=training_args,
        tokenizer=tokenizer,
    )

<ipython-input-22-5ee9dc9ddd78>:9: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=63, training_loss=2.3513256254650297, metrics={'train_runtime': 539.2215, 'train_samples_per_second': 0.929, 'train_steps_per_second': 0.117, 'total_flos': 1814352831184896.0, 'train_loss': 2.3513256254650297, 'epoch': 3.0})

In [27]:
finetuned_checkpoint = "/content/drive/MyDrive/GIL/Unlearning/models/olmo-finetuning"
trainer.save_model(finetuned_checkpoint)

In [31]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/GIL/Unlearning/models/olmo-finetuning")
torch.save(model, "/content/drive/MyDrive/GIL/Unlearning/models/olmo-finetuning.pt")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Task vector

In [32]:
import torch


class TaskVector():
    def __init__(self, pretrained_checkpoint=None, finetuned_checkpoint=None, vector=None):
        """Initializes the task vector from a pretrained and a finetuned checkpoints.

        This can either be done by passing two state dicts (one corresponding to the
        pretrained model, and another to the finetuned model), or by directly passying in
        the task vector state dict.
        """
        if vector is not None:
            self.vector = vector
        else:
            assert pretrained_checkpoint is not None and finetuned_checkpoint is not None
            with torch.no_grad():
                pretrained_state_dict = torch.load(pretrained_checkpoint).state_dict()
                finetuned_state_dict = torch.load(finetuned_checkpoint).state_dict()
                self.vector = {}
                for key in pretrained_state_dict:
                    if pretrained_state_dict[key].dtype in [torch.int64, torch.uint8]:
                        continue
                    self.vector[key] = finetuned_state_dict[key] - pretrained_state_dict[key]

    def __add__(self, other):
        """Add two task vectors together."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                if key not in other.vector:
                    print(f'Warning, key {key} is not present in both task vectors.')
                    continue
                new_vector[key] = self.vector[key] + other.vector[key]
        return TaskVector(vector=new_vector)

    def __radd__(self, other):
        if other is None or isinstance(other, int):
            return self
        return self.__add__(other)

    def __neg__(self):
        """Negate a task vector."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                new_vector[key] = - self.vector[key]
        return TaskVector(vector=new_vector)

    def apply_to(self, pretrained_checkpoint, scaling_coef=1.0):
        """Apply a task vector to a pretrained model."""
        with torch.no_grad():
            pretrained_model = torch.load(pretrained_checkpoint)
            new_state_dict = {}
            pretrained_state_dict = pretrained_model.state_dict()
            for key in pretrained_state_dict:
                if key not in self.vector:
                    print(f'Warning: key {key} is present in the pretrained state dict but not in the task vector')
                    continue
                new_state_dict[key] = pretrained_state_dict[key] + scaling_coef * self.vector[key]
        pretrained_model.load_state_dict(new_state_dict, strict=False)
        return pretrained_model

In [37]:
import torch


class TaskVector():
    def __init__(self, pretrained_checkpoint=None, finetuned_checkpoint=None, vector=None, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """Initializes the task vector from a pretrained and a finetuned checkpoints.

        This can either be done by passing two state dicts (one corresponding to the
        pretrained model, and another to the finetuned model), or by directly passying in
        the task vector state dict.
        """
        self.device = device  # Added device attribute
        if vector is not None:
            self.vector = vector
        else:
            assert pretrained_checkpoint is not None and finetuned_checkpoint is not None
            with torch.no_grad():
                pretrained_state_dict = torch.load(pretrained_checkpoint, map_location=self.device).state_dict()  # Load to specified device
                finetuned_state_dict = torch.load(finetuned_checkpoint, map_location=self.device).state_dict()  # Load to specified device
                self.vector = {}
                for key in pretrained_state_dict:
                    # Filter out quantization-related keys
                    if any(quant_key in key for quant_key in ['.absmax', '.code']):
                        continue

                    if pretrained_state_dict[key].dtype in [torch.int64, torch.uint8]:
                        continue

                    #Check if the key exists in the finetuned state dictionary
                    if key in finetuned_state_dict:
                        self.vector[key] = finetuned_state_dict[key] - pretrained_state_dict[key]
                    else:
                        print(f'Warning: key {key} is present in the pretrained state dict but not in the finetuned state dict')


    def __add__(self, other):
        """Add two task vectors together."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                if key not in other.vector:
                    print(f'Warning, key {key} is not present in both task vectors.')
                    continue
                new_vector[key] = self.vector[key] + other.vector[key]
        return TaskVector(vector=new_vector, device=self.device)  # Pass device to new instance

    def __radd__(self, other):
        if other is None or isinstance(other, int):
            return self
        return self.__add__(other)

    def __neg__(self):
        """Negate a task vector."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                new_vector[key] = - self.vector[key]
        return TaskVector(vector=new_vector, device=self.device)  # Pass device to new instance

    def apply_to(self, pretrained_checkpoint, scaling_coef=1.0):
        """Apply a task vector to a pretrained model."""
        with torch.no_grad():
            pretrained_model = torch.load(pretrained_checkpoint, map_location=self.device)  # Load to specified device
            new_state_dict = {}
            pretrained_state_dict = pretrained_model.state_dict()
            for key in pretrained_state_dict:
                if key not in self.vector:
                    print(f'Warning: key {key} is present in the pretrained state dict but not in the task vector')
                    continue
                new_state_dict[key] = pretrained_state_dict[key] + scaling_coef * self.vector[key]
        pretrained_model.load_state_dict(new_state_dict, strict=False)
        return pretrained_model

In [38]:
pretrained_checkpoint = "/content/drive/MyDrive/GIL/Unlearning/models/olmo.pt"
finetuned_checkpoint = "/content/drive/MyDrive/GIL/Unlearning/models/olmo-finetuning.pt"
# Create the task vector
task_vector = TaskVector(pretrained_checkpoint, finetuned_checkpoint)
# Negate the task vector
neg_task_vector = -task_vector
# Apply the task vector
model_after_vector = neg_task_vector.apply_to(pretrained_checkpoint, scaling_coef=0.5)
#save model
final_model = "/content/drive/MyDrive/GIL/Unlearning/models/olmo-vector.pt"
torch.save(model_after_vector, final_model)

<ipython-input-37-8b4880037138>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state_dict = torch.load(pretrained_checkpoint, map_location=self.device).state_d

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 63.06 MiB is free. Process 5310 has 14.68 GiB memory in use. Of the allocated memory 14.52 GiB is allocated by PyTorch, and 28.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)